In [ ]:
! mkdir -p datasets
%cd datasets
! wget -nc https://raw.githubusercontent.com/pablonoya/zigzag-ml/master/datasets/housing.csv
%cd ..

# ¿Por qué crear features?
Como mencionamos, la **elección de features** influye en el rendimento del modelo, y podemos encontrarnos en situaciones como tener medidas separadas de largo y ancho que podemos multiplicar para tener una sóla feature: el área, la cual será más fácil de manejar y representará a dos features a la vez.

Y esta idea de usar la **multiplicación de features** puede ayudarnos en una situación aún más común, mira el siguiente gráfico que relaciona dos columnas de nuestro dataset.

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt

data_housing = pd.read_csv('./datasets/housing.csv')
data_housing.dropna(inplace=True)
data_housing.head()

In [ ]:
import matplotlib.pyplot as plt

X = data_housing['total_rooms']
y = data_housing['housing_median_age']

plt.figure(figsize=(8, 6))
plt.scatter(X, y)
plt.xlabel('total rooms')
plt.ylabel('housing median age')

¿Puede una **recta ajustar estos datos** ?  
Toma en cuenta que existen varios valores en el eje x, como si fueran otra recta.

Si tuviéramos un trazo que baja de izquierda a derecha acercándose a la esquina inferior izquierda... ¡tendríamos una curva!

# Regresión polinomial
Podemos describir una curva con la siguiente ecuación, que a su vez describe a un polinomio:

$P(x) = a_nx^n + a_{n-1}x^{n-1} + \cdots + a_0x^0$

Notarás que se **parece a la regresión lineal múltiple**, con una pequeña gran diferencia, se usa la **misma feature elevada a un rango de exponentes**, que va de cero a un número **n**.

Mientras más alto sea este número, denominado grado, más sinuosa será la gráfica.  
![curves](https://www.themathpage.com/aPreCalc/Pre_Img/B12.png)

Este procesamiento se conoce como **regresión polinomial**, probemos elevando a dos, incluyendo la misma feature al cuadrado.  
Podemos "apilar" varios arrays contenidos en una tupla utilizando `column_stack` de numpy, esto tratará cada array como una columna, pero es necesario que todos los arrays tengan la misma longitud.

In [ ]:
from sklearn.model_selection import train_test_split
import numpy as np

np.random.seed(42)

X_2D = X.values.reshape(-1, 1)
X_train, X_test, y_train, y_test = train_test_split(X_2D, y, test_size=0.2)

X_train_poly = np.column_stack((X_train ** 2, X_train))
X_train_poly[:5]

Ahora entrenemos un modelo, como si fuera una regresión lineal múltiple

In [ ]:
from sklearn.linear_model import LinearRegression

model_poly = LinearRegression()
model_poly.fit(X_train_poly, y_train)

## Veamos la curva
Si unimos sólo 2 puntos tendremos una recta aunque usemos la ecuación polinómica de `model_poly`.

In [ ]:
xpoints = np.array([min(X_train), max(X_train)])
xpoints_poly = np.column_stack((xpoints ** 2, xpoints))

ypoints = model_poly.predict(xpoints_poly)

plt.scatter(X, y)
plt.plot(xpoints, ypoints, color="red")

Necesitamos valores intermedios, mientras más tengamos, más suave será nuestra curva. La función `linspace` de numpy nos permite generar $n$ **valores equidistantes** entre sí dentro de un rango definido.

Generemos 5 valores, el rango será el mínimo y el máximo de X_train, que contiene los valores originales, sin elevar al cuadrado.

In [ ]:
xpoints = np.linspace(start=min(X_train), stop=max(X_train), num=5)
xpoints

Utilizemos estos puntos intermedios para graficar la curva

In [ ]:
xpoints_poly = np.column_stack((xpoints ** 2, xpoints))
ypoints = model_poly.predict(xpoints_poly)

plt.plot(xpoints, ypoints, color="red")

Si deseamos suavizar la curva no es necesario generar **demasiados** valores.

In [ ]:
xpoints = np.linspace(start=min(X_train), stop=max(X_train), num=20)
xpoints_poly = np.column_stack((xpoints ** 2, xpoints))
ypoints = model_poly.predict(xpoints_poly)

plt.plot(xpoints, ypoints, color="red")
plt.scatter(X_train, y_train)

## Genera polinomios
El objeto [PolynomialFeatures](https://scikit-learn.org/stable/modules/generated/sklearn.preprocessing.PolynomialFeatures.html?highlight=polynomial#sklearn-preprocessing-polynomialfeatures) de sklearn convierte las features en polinomios, el parámetro `degree` indica el [grado](https://es.wikipedia.org/wiki/Grado_(polinomio)), esto es, el exponente máximo al que elevaremos.

Su método `fit_transform` nos devolverá una matriz con cada fila en el formato $x^0, x^1, \cdots, x^n$  si el parámetro es **una** sola feature.  
De manera interna, este método llama primero a `fit` para calcular qué features tendremos y luego a `transform` para crear dichas features, esta separación es importante en otro tipo de procedimientos, pues sólo deberíamos usar el conjunto de entrenamiento para realizar ajustes y el de prueba para las pruebas.

Como $x^0 = 1$ tendremos una columna llena de unos, este término se conoce como **bias** o sesgo, ¡un termino que será muy importante!, pero que ahora excluiremos estableciendo `include_bias` en falso.

In [ ]:
from sklearn.preprocessing import PolynomialFeatures

poly = PolynomialFeatures(degree=3, include_bias=False)
X_train_poly = poly.fit_transform(X_train)
X_train_poly[:5]

Si decidimos incluir varias features tendremos un **polinomio de varias variales**.  
Veamos uno de **dos variables y grado dos**, sin coeficientes:

${x_1}^2 + x_1 + x_1x_2 + x_2 +{x_2}^2$

El exponente máximo es dos, aunque el término del medio tenga las dos variables, la suma de sus exponentes es 2 debido a que ambos tienen grado 1.  
Así, un término con la forma ${x_1}^2x_2$ tendría grado 3, por la suma de sus exponentes.

En nuestro caso de dos variables y grado 2 `PolynomialFeatures` devolverá una lista con el formato $x_1, x_2, {x_1}^2, x_1x_2, {x_2}^2$

In [ ]:
# cambiamos de grado
poly = PolynomialFeatures(degree=2, include_bias=False)

# tomamos dos features
X_selected = data_housing[['total_rooms', 'total_bedrooms']]

# recuerda que sólo tomamos una parte del dataset
X_train2 = X_selected[:len(X_train)]

X_train_poly = poly.fit_transform(X_train2)
X_train_poly[:5]

## ¡Pero no generes tantos!
Prueba aumentar el grado, verás más columnas, y muchas más si añades features antes de generar polinomios 😱.

Estas columnas cuentan como nuevas features para el modelo, nuevas features pueden disminuir el error, pero por cada columna **añades una dimensión** y esto causa un problema conocido como *curse of dimensionality* o maldición de la dimensión.

Esto trae "problemas que no se ven en menores dimensiones", como la **dispesión** y la **cercanía** de los datos.  

Lo explicaré mejor con un pequeño juego imaginario.
Ahoras estás en el principio de un espacio de 4 secciones y debes debes alcanzar un objeto que está al final:

    O--*
    
Sólo debes recorrer 3 secciones para alcanzarlo, estás muy cerca del objeto, lo alcanzas y pasas de nivel.

Ahora añadimos otra dimensión, tienes un espacio de 3 x 3, con 9 secciones en total:

    XX*
    XXX
    OXX

Ahora debes recorrer 4 secciones para alcanzar el objeto, ya no está tan cerca pero no tardas en alcanzarlo y pasar a otro nivel.

Se añade otra dimensión también con 3 secciones, tendemos un juego 3D con 27 secciones y la misma situación.
¿Cuántas secciones debes recorrer? 

Al añadir dimensiones, tú y el objeto se han **dispersado**, perdiendo **cercanía**, lo mismo pasa con los datos cuando añadimos features, será más dificil para el modelo encontrar la recta que mejor se ajuste porque tenemos distancias más grandes entre los puntos.

## A menos que...
Si contamos con **más datos** de los que aprender, un modelo **más complejo** rendirá mejor, una mayor cantidad de datos nos ayudará a ver un panorama más general. Este aspecto lo trataremos mejor más adelante 😉.

# Probando la curva
Veamos si el modelo polinomial se desempeña mejor al final, para esto es necesario transformar también el test set, usaremos sólo `transform` pues el ajuste que corresponde a la función `fit` se debe realizar en el conjunto de entrenamiento.

In [ ]:
from sklearn.metrics import mean_squared_error

poly = PolynomialFeatures(degree=2, include_bias=False)

X_train_poly = poly.fit_transform(X_train)
X_test_poly = poly.transform(X_test)

model_poly.fit(X_train_poly, y_train)

y_hat_poly_train = model_poly.predict(X_train_poly)
y_hat_poly = model_poly.predict(X_test_poly)

print("MSE train poly", mean_squared_error(y_hat_poly_train, y_train))
print("MSE test poly", mean_squared_error(y_hat_poly, y_test))

Para comparar, entrenamos un modelo de regresión lineal simple

In [ ]:
model = LinearRegression()
model.fit(X_train, y_train)

y_hat_train = model.predict(X_train)
y_hat = model.predict(X_test)

print("MSE train", mean_squared_error(y_hat_train, y_train))
print("MSE test", mean_squared_error(y_hat, y_test))

# Ejercicios
Prueba generar polinomios con grados más altos y ejecuta las pruebas, ¿las métricas mejoran?

In [ ]:
# sólo cambia un parámetro


Desde el principio no incluímos nuestra columna objetivo, la del precio, inclúyela junto a una o más features para generar un poliniomio que mejore el modelo del capítulo anterior 😎.

In [ ]:
# cambia y, divide en conjuntos y entrena con los polinomios


Es cierto que un modelo más complejo permite aprender mejor, pero si aprendes mejor sólo unas cuantas cosas... estarías perdiendo la idea general.  
Lo mismo pasa con machine learning, es importante que lo aprendido se pueda **generalizar**, que el conocimiento funcione no sólo con lo que hemos visto, de lo contrario tenemos un problema llamado **overfitting** o [sobreajuste](./5_overfitting.ipynb).